DATA PREPARATION

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import cv2

BASE_DIR = Path.cwd()
DATASET_PATH = BASE_DIR / "dataset_filtered"
LABEL_MAP = BASE_DIR / "label_map.txt"
MODEL_NAME = BASE_DIR / "crop_disease_model.keras"
EPOCHS = 12

In [2]:
if not DATASET_PATH.exists():
    raise FileNotFoundError(
        f"Dataset folder not found at {DATASET_PATH}\n"
        "Please download the dataset and place it in the project root as 'dataset_filtered/'"
    )

In [3]:
train_datagen = ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [4]:
val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

In [5]:
train_data = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

Found 16254 images belonging to 13 classes.


In [6]:
val_data = val_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

Found 4058 images belonging to 13 classes.


In [7]:
NUM_CLASSES = train_data.num_classes
labels = train_data.class_indices
with open(LABEL_MAP, "w") as f:
    for label, idx in labels.items():
        f.write(f"{idx}:{label}\n")

print("✅ Label map saved to", LABEL_MAP)

✅ Label map saved to d:\Lohith\BTECH CODES\Machine_Learning\Project\Crop-Disease\label_map.txt


MODEL ARCHITECTURE AND TRAINING

In [8]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False 

In [9]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
])

In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 13)             │         3,341 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,589,261 (9.88 MB)

 Trainable params: 331,277 (1.26 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [12]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_data.classes),
    y=train_data.classes
)

class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

Class Weights: {0: np.float64(1.5628846153846154), 1: np.float64(1.5628846153846154), 2: np.float64(10.2484237074401), 3: np.float64(0.7346108650456477), 4: np.float64(1.5628846153846154), 5: np.float64(0.8182641965364479), 6: np.float64(1.6408237431859478), 7: np.float64(0.8823625210357744), 8: np.float64(0.9323696437790397), 9: np.float64(1.1123733917328222), 10: np.float64(0.29171901360422126), 11: np.float64(4.181631077952148), 12: np.float64(0.9821741494954378)}


In [13]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(MODEL_NAME, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3)
]

In [14]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=callbacks
)
print("Final Training Accuracy:", history.history["accuracy"][-1])

Epoch 1/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 407s 796ms/step - accuracy: 0.6634 - loss: 1.1093 - val_accuracy: 0.8376 - val_loss: 0.5336 - learning_rate: 3.0000e-04
Epoch 2/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 195s 384ms/step - accuracy: 0.8123 - loss: 0.5804 - val_accuracy: 0.8509 - val_loss: 0.4403 - learning_rate: 3.0000e-04
Epoch 3/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 196s 386ms/step - accuracy: 0.8452 - loss: 0.4797 - val_accuracy: 0.8455 - val_loss: 0.4345 - learning_rate: 3.0000e-04
Epoch 4/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 197s 387ms/step - accuracy: 0.8627 - loss: 0.4217 - val_accuracy: 0.8825 - val_loss: 0.3319 - learning_rate: 3.0000e-04
Epoch 5/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 196s 387ms/step - accuracy: 0.8779 - loss: 0.3777 - val_accuracy: 0.8797 - val_loss: 0.3457 - learning_rate: 3.0000e-04
Epoch 6/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 197s 389ms/step - accuracy: 0.8848 - loss: 0.3526 - val_accuracy: 0.8879 - val_loss: 0.3226 - learning_rate: 3.0000e-04
Epoch 7/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 

In [26]:
model.save(MODEL_NAME)
print("✅ Final model saved")


✅ Final model saved


In [27]:
label_map = {}
with open(LABEL_MAP, "r") as f:
    for line in f:
        idx, label = line.strip().split(":")
        label_map[int(idx)] = label

In [37]:
REMEDIATION_DB = {
    "Potato___Early_blight": {
        "organic": [
            "Remove infected leaves",
            "Spray neem oil (3 ml/L)",
            "Improve air circulation"
        ],
        "chemical": [
            "Spray Mancozeb (2.5 g/L)",
            "Repeat every 7 days"
        ],
        "prevention": [
            "Avoid overhead irrigation",
            "Use disease-resistant varieties",
            "Crop rotation"
        ],
        "safety": [
            "Wear gloves & mask",
            "Do not spray during noon",
            "Keep away from children"
        ]
    },

    "Potato___Late_blight": {
        "organic": [
            "Destroy infected plants",
            "Apply compost tea"
        ],
        "chemical": [
            "Spray Metalaxyl (2 g/L)",
            "Repeat every 5 days"
        ],
        "prevention": [
            "Avoid water stagnation",
            "Use certified seeds"
        ],
        "safety": [
            "Use PPE",
            "Avoid spraying before rain"
        ]
    },

    "Tomato___Early_blight": {
        "organic": [
            "Remove affected leaves",
            "Use neem oil spray"
        ],
        "chemical": [
            "Spray Chlorothalonil",
            "Repeat every 7 days"
        ],
        "prevention": [
            "Mulching",
            "Avoid wet leaves"
        ],
        "safety": [
            "Avoid inhalation",
            "Wear gloves"
        ]
    }
}

In [28]:
model = tf.keras.models.load_model(MODEL_NAME)

dummy_input = np.zeros((1, 224, 224, 3), dtype=np.float32)
model(dummy_input)  # 👈 IMPORTANT: call model directly, not predict()


<tf.Tensor: shape=(1, 13), dtype=float32, numpy=
array([[1.03741854e-07, 2.15192676e-06, 6.48439936e-07, 2.08785437e-04,
        7.98078690e-05, 9.97325420e-01, 6.60226988e-06, 1.92963460e-03,
        2.79244932e-06, 1.59534375e-06, 2.55323044e-04, 6.22783148e-07,
        1.86423902e-04]], dtype=float32)>

In [29]:
def is_blurry(img_path):
    img = cv2.imread(img_path)
    variance = cv2.Laplacian(img, cv2.CV_64F).var()
    threshold = 100.0  # You can adjust this threshold
    return variance < threshold

In [30]:
def generate_gradcam(model, img_path, layer_name=None):
    """Generate a Grad-CAM heatmap overlay for the predicted class.

    Fixes Keras graph issues by building the Grad-CAM model using the
    MobileNetV2 base model's input graph (not Sequential's symbolic tensors).
    """
    base_model = model.layers[0]  # MobileNetV2 (Functional)

    # Pick a sensible conv layer if not provided
    if layer_name is None:
        for layer in reversed(base_model.layers):
            if isinstance(layer, tf.keras.layers.Conv2D):
                layer_name = layer.name
                break
        if layer_name is None:
            # Fallback: any layer with 'conv' in its name
            for layer in reversed(base_model.layers):
                if "conv" in layer.name.lower():
                    layer_name = layer.name
                    break

    if layer_name is None:
        raise ValueError("No convolutional layer found in base model.")

    conv_layer = base_model.get_layer(layer_name)

    # Build a Grad-CAM graph rooted at base_model.input to avoid tensor mismatch
    x = base_model.output
    for head_layer in model.layers[1:]:
        x = head_layer(x)
    preds = x

    grad_model = tf.keras.models.Model(
        inputs=base_model.input,
        outputs=[conv_layer.output, preds],
    )

    # Load and preprocess image
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0).astype(np.float32)

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(pooled_grads * conv_outputs, axis=-1)

    # Convert to numpy once, then use numpy/cv2 safely
    heatmap = heatmap.numpy()
    heatmap = np.maximum(heatmap, 0)
    heatmap /= (np.max(heatmap) + 1e-8)

    heatmap = cv2.resize(heatmap, (224, 224))
    heatmap = np.uint8(255 * heatmap)

    original = cv2.imread(img_path)
    if original is None:
        raise FileNotFoundError(f"Could not read image from path: {img_path}")
    original = cv2.resize(original, (224, 224))

    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(original, 0.6, heatmap, 0.4, 0)

    output_path = "heatmap_output.jpg"
    cv2.imwrite(output_path, overlay)
    return output_path

In [ ]:
def predict_disease(img_path, confidence_threshold=0.45):
    if is_blurry(img_path):
        return {
            "status": "error",
            "message": "Image is too blurry. Please retake photo.",
            "confidence": 0.0
        }
    
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)[0]
    top_idx = np.argmax(preds)
    confidence = float(preds[top_idx])
    disease_name = label_map[top_idx]

    # Epic 3: Unknown/Unclear Result Story
    if confidence < confidence_threshold:
        return {
            "status": "unknown",
            "disease": "Unknown/Unclear",
            "message": "Please consult a human expert or retake the photo.",
            "confidence": round(confidence * 100, 2)
        }
    
    # Epic 3: Healthy Plant Story
    # Check if the predicted label contains the word "healthy"
    is_healthy = "healthy" in disease_name.lower()

    return {
        "status": "success",
        "disease": disease_name,
        "is_healthy": is_healthy,
        "confidence": round(confidence * 100, 2)
    }

In [32]:
print("Final Training Accuracy:", history.history["accuracy"][-1])
print("Final Validation Accuracy:", history.history["val_accuracy"][-1])


Final Training Accuracy: 0.921373188495636
Final Validation Accuracy: 0.9139970541000366


In [33]:
def predict_top_k(img_path, k=3, min_conf=0.05):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)[0]
    top_indices = preds.argsort()[-k:][::-1]

    results = []
    for idx in top_indices:
        conf = float(preds[idx])
        if conf >= min_conf:
            results.append({
                "disease": label_map[idx],
                "confidence": round(conf * 100, 2)
            })

    return results


In [34]:
def estimate_severity_from_heatmap(heatmap_path):
    heatmap = cv2.imread(heatmap_path)
    heatmap_gray = cv2.cvtColor(heatmap, cv2.COLOR_BGR2GRAY)

    # Threshold to isolate "hot" infected zones
    _, binary_map = cv2.threshold(heatmap_gray, 200, 255, cv2.THRESH_BINARY)

    infected_area = np.sum(binary_map == 255)
    total_area = binary_map.shape[0] * binary_map.shape[1]

    ratio = infected_area / total_area

    if ratio < 0.10:
        severity = "Low"
    elif ratio < 0.40:
        severity = "Medium"
    else:
        severity = "High"

    return {
        "severity": severity,
        "infected_ratio": round(ratio * 100, 2)
    }


In [40]:
def get_remediation_plan(disease, severity):
    if disease not in REMEDIATION_DB:
        return {
            "message": "No remediation data available for this disease.",
            "escalate": True
        }

    data = REMEDIATION_DB[disease]

    if severity == "Low":
        treatment = data["organic"]
        urgency = "Monitor & apply organic remedies."
    elif severity == "Medium":
        treatment = data["organic"] + data["chemical"]
        urgency = "Apply combined treatment."
    else:
        treatment = data["chemical"]
        urgency = "Immediate chemical treatment required."

    escalate = severity == "High"

    return {
        "urgency": urgency,
        "treatment_steps": treatment,
        "prevention_tips": data["prevention"],
        "safety_warnings": data["safety"],
        "escalate_to_expert": escalate
    }


In [41]:
def full_predict_with_heatmap(img_path):
    result = predict_disease(img_path)

    if result["status"] != "success":
        return result

    heatmap_path = generate_gradcam(model, img_path)
    severity_info = estimate_severity_from_heatmap(heatmap_path)

    remediation = get_remediation_plan(
        result["disease"],
        severity_info["severity"]
    )

    result["heatmap"] = heatmap_path
    result["severity"] = severity_info["severity"]
    result["infected_ratio"] = severity_info["infected_ratio"]
    result["remediation"] = remediation

    return result


In [43]:
def pretty_print_result(result):
    print("\n========== AI CROP DIAGNOSIS REPORT ==========\n")

    print(f"Status           : {result.get('status')}")
    print(f"Disease          : {result.get('disease')}")
    print(f"Healthy          : {result.get('is_healthy')}")
    print(f"Confidence       : {result.get('confidence')} %")
    print(f"Severity         : {result.get('severity')}")
    print(f"Infected Ratio   : {result.get('infected_ratio')} %")

    if "remediation" in result:
        r = result["remediation"]

        print("\n--- Remediation Plan ---")
        print(f"Urgency          : {r.get('urgency')}")

        print("\nTreatment Steps:")
        for i, step in enumerate(r.get("treatment_steps", []), 1):
            print(f"  {i}. {step}")

        print("\nPrevention Tips:")
        for tip in r.get("prevention_tips", []):
            print(f"  - {tip}")

        print("\nSafety Warnings:")
        for warn in r.get("safety_warnings", []):
            print(f"  ⚠ {warn}")

        print(f"\nEscalate to Expert : {'Yes' if r.get('escalate_to_expert') else 'No'}")

    print("\n=============================================\n")


In [ ]:
if __name__ == "__main__":
    TEST_IMAGE = r'0a8a68ee-f587-4dea-beec-79d02e7d3fa4___RS_Early.B 8461.JPG'

    result = full_predict_with_heatmap(TEST_IMAGE)
    pretty_print_result(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

--- Final Prediction with Severity ---
{'status': 'success', 'disease': 'Potato___Early_blight', 'is_healthy': False, 'confidence': 99.99, 'heatmap': 'heatmap_output.jpg', 'severity': 'Low', 'infected_ratio': np.float64(0.69), 'remediation': {'urgency': 'Monitor & apply organic remedies.', 'treatment_steps': ['Remove infected leaves', 'Spray neem oil (3 ml/L)', 'Improve air circulation'], 'prevention_tips': ['Avoid overhead irrigation', 'Use disease-resistant varieties', 'Crop rotation'], 'safety_warnings': ['Wear gloves & mask', 'Do not spray during noon', 'Keep away from children'], 'escalate_to_expert': False}}
